## Overview
This notebook builds a clean movie-person dataset from IMDb TSV exports. The steps below load raw files, filter to relevant rows, merge the tables together, tidy repeated-string columns, and finally write a CSV for downstream modeling.

## Setup and imports
Import pandas and math; helper functions live in the next cell.

In [1]:
import pandas as pd

## Helper function
`tsv_to_dataframe` streams TSV files in chunks, with optional column filtering, to avoid loading the full file in memory at once.

In [2]:
def tsv_to_dataframe(path, filter_column = '', filter_value = ''):
    chunks = []
    for chunk in pd.read_csv(filepath_or_buffer=path, sep='\t', chunksize=10000):
        if len(filter_column) > 0 and len(filter_value) > 0:
            filtered_chunk = chunk[chunk[filter_column] == filter_value]
        else:
            filtered_chunk = chunk
        chunks.append(filtered_chunk)
    return pd.concat(chunks, ignore_index=True)

## Load source TSV files
Define the IMDb data file locations (both forward- and backslash variants) before merging them into one dataset.

### Filepaths
Toggle `useBackslash` if running on Windows; otherwise default to forward slashes.

In [ ]:
useBackslash = False

if not useBackslash:
    title_path = r'../data/title.basics.tsv'
    crew_path = r'../data/title.crew.tsv'
    ratings_path = r'../data/title.ratings.tsv'
    principals_path = r'../data/title.principals.tsv'
    names_path = r'../data/name.basics.tsv'
    alternate_path = r'../data/title.akas.tsv'
else:
    title_path = r'..\data\title.basics.tsv'
    crew_path = r'..\data\title.crew.tsv'
    ratings_path = r'..\data\title.ratings.tsv'
    principals_path = r'..\data\title.principals.tsv'
    names_path = r'..\data\name.basics.tsv'
    alternate_path = r'..\data\title.akas.tsv'

### Merge TSVs into one dataset
- Filter titles to movies only; keep ratings with at least 100 votes to avoid noise.
- Inner joins ensure we retain only rows present across the linked tables.

In [ ]:
title = tsv_to_dataframe(path=title_path, filter_column='titleType', filter_value='movie')
crew = tsv_to_dataframe(path=crew_path)
dataset = pd.merge(title, crew, on='tconst', how='inner')
del crew, title

In [5]:
ratings = tsv_to_dataframe(path=ratings_path)
dataset = pd.merge(dataset, ratings, on='tconst', how='inner')
del ratings

In [6]:
principals = tsv_to_dataframe(path=principals_path)
dataset = pd.merge(dataset, principals, on='tconst', how='inner')
del principals

In [7]:
names = tsv_to_dataframe(path=names_path)
dataset = pd.merge(dataset, names, how='inner', on='nconst')
del names

## Expand comma-separated columns
Split `knownForTitles` and `genres` into separate columns for easier analysis, then drop the original wide columns.

In [ ]:
dataset[['known_for_movie_1', 'known_for_movie_2', 'known_for_movie_3', 'known_for_movie_4']] = dataset['knownForTitles'].str.split(',', expand=True)
dataset[['genre_1', 'genre_2', 'genre_3']] = dataset['genres'].str.split(',', expand=True)

In [7]:
dataset.drop('knownForTitles', axis=1, inplace=True)
dataset.drop('endYear', axis=1, inplace=True)
dataset.drop('genres', axis=1, inplace=True)

## Save the prepared dataset
Export the merged and tidied dataframe to CSV for later notebooks.

In [ ]:
locateFileAt = r'..\data\merging_tsv_files_data.csv' if useBackslash  else r'../data/merging_tsv_files_data.csv'
dataset.to_csv(locateFileAt, sep =';')